In [1]:
# Añado al sys.path el directorio raíz:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

import pandas as pd
import pandasql as ps
import matplotlib.pyplot as plt
from modulos.funciones import fun_con

# Establezco configuración
pd.set_option('display.max_columns', None) # all columns are shown
pd.set_option('display.max_colwidth', None) # all cell content is shown

with open('nutritions.sql', 'r') as file:
    query = file.read()

df = fun_con(query)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   episode_ref       57 non-null     int64         
 1   start_date        57 non-null     datetime64[ns]
 2   start_drug_date   57 non-null     datetime64[ns]
 3   drug_descr        57 non-null     object        
 4   hours_difference  57 non-null     int64         
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 2.4+ KB


In [2]:
start_timeframe = input("Input the timeframe to analyze in SQL format 2023-07-12")
end_timeframe = input("Input the timeframe to analyze in SQL format 2023-07-12")
unit1 = input("Enter the unit you want to analyze")
unit2 = input("Enter the second unit you want to analyze")


In [3]:
query = f"""

WITH movement_data AS (
    SELECT 
        m.patient_ref,
        m.episode_ref,
        m.start_date,
        m.end_date,
        TIMESTAMPDIFF(hour, m.start_date, m.end_date) AS horesingres,
        LEAD(m.start_date) OVER (PARTITION BY m.episode_ref ORDER BY m.start_date) AS next_start_date
    FROM
        g_mov_events AS m
    WHERE 
        ou_loc_ref IN ('{unit1}', '{unit2}') 
        AND start_date BETWEEN '{start_timeframe} 00:00:00' AND '{end_timeframe} 23:59:59'
        AND m.start_date != m.end_date
        AND TIMESTAMPDIFF(hour, m.start_date, m.end_date) > 1
),
fc AS (
    SELECT *
    FROM movement_data
    WHERE next_start_date IS NULL
       OR end_date != next_start_date
),
fc_with_drugs AS (
    SELECT 
        fc.episode_ref, 
        fc.start_date, 
        pe.start_drug_date, 
        pe.drug_descr,
        ROW_NUMBER() OVER (PARTITION BY fc.episode_ref ORDER BY pe.start_drug_date ASC) AS rn
    FROM 
        fc
    INNER JOIN 
        g_prescription_events AS pe
    ON 
        fc.episode_ref = pe.episode_ref
    WHERE 
        pe.drug_descr IN ('NUTRICION ENTERAL', 'NUTRICIÓN PARENTERAL CENTRAL')
)
SELECT 
    episode_ref, 
    start_date, 
    start_drug_date, 
    drug_descr, 
    TIMESTAMPDIFF(hour, start_date, start_drug_date) AS hours_difference
FROM 
    fc_with_drugs
WHERE 
    rn = 1
    AND TIMESTAMPDIFF(hour, start_date, start_drug_date) > 0
ORDER BY 
    episode_ref, start_date;

"""

df = fun_con(query)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   episode_ref       57 non-null     int64         
 1   start_date        57 non-null     datetime64[ns]
 2   start_drug_date   57 non-null     datetime64[ns]
 3   drug_descr        57 non-null     object        
 4   hours_difference  57 non-null     int64         
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 2.4+ KB


In [4]:
enterals = df.loc[df['drug_descr'] == 'NUTRICION ENTERAL', 'drug_descr'].count()
parenterals = df.loc[df['drug_descr'] == 'NUTRICIÓN PARENTERAL CENTRAL', 'drug_descr'].count()
nutri_totals = len(df)
perc_enterals = enterals / nutri_totals *100
perc_parenterals = parenterals / nutri_totals *100
print(f"El percentatge de nutris enterals és de {perc_enterals.round(2)} %")
print("El percentatge de nutris parenterals és de {} %".format(perc_parenterals.round(2)))

El percentatge de nutris enterals és de 43.86 %
El percentatge de nutris parenterals és de 56.14 %


In [5]:
temps_enteral = df.loc[df['drug_descr'] == 'NUTRICION ENTERAL', 'hours_difference'].mean()
temps_parenteral = df.loc[df['drug_descr'] == 'NUTRICIÓN PARENTERAL CENTRAL', 'hours_difference'].mean()
print("El temps fins l'inici de la nutrició enteral és de {} hores".format(temps_enteral.round()))
print("El temps fins l'inici de la nutrició parenteral és de {} hores".format(temps_parenteral.round()))

El temps fins l'inici de la nutrició enteral és de 88.0 hores
El temps fins l'inici de la nutrició parenteral és de 95.0 hores
